# Get Standup Show Ratings

# IMBD API

## Imports

In [206]:
import requests
import numpy as np
import pandas as pd
import re
import json

import pickle

import plotly.express as px

In [169]:
tscripts = pd.read_pickle('../data/transcripts_raw_df.pickle')
tscripts['script characters'] = tscripts['transcript'].apply(len)
px.strip(tscripts, x = 'script characters', hover_data= ['description'])


In [173]:
tscripts = tscripts[tscripts['script characters'] > 21000]
shows_split = tscripts['description'].str.split('|', expand = True)[0]

In [175]:
def transcript_strip(string):
    hasyear = re.search(r"\(\d{4}\)", string)
    hasdashbracket = re.search(r"\s[–\[]", string)
    if hasyear:
        return string[:hasyear.end()]
    elif hasdashbracket:
        return string[:hasdashbracket.start()]
    else:
        return string

In [177]:
show_searches =  [transcript_strip(search) for search in shows_split]

## Get Ratings from API
I'll have to only do 100 results per day since I'm sticking with the free version of the API.
After I get all the title IDs (4 days), I'll then have to get the ratings (another 4 days). If I want the reviews, that's another 4 days.

In [181]:
api_key = json.load(open('../../../api_keys/api_keys.json'))['imdb']

In [182]:
search_results = []
search_results_fails = []

In [226]:
# ended at 100 on 5/20

with open('../data/imdb_title_search_results.pickle', 'rb') as file:
    search_results = pickle.load(file)

with open('../data/imdb_title_search_results_fails.pickle', 'rb') as file:
    search_results_fails = pickle.load(file)

start = 75
end = 100
for search in show_searches[start:end]:

    try:
        endpoint = 'https://imdb-api.com/en/API/SearchTitle/'
        params = {
            'apiKey':api_key,
            'expression':search
        }
        response = requests.get(endpoint, params)
        result = response.json()

        search_result = result['results'][0]
        search_results.append(search_result)
    except:
        search_results_fails.append(search)

In [227]:
search_results_fails

['Andy Woodhull: You’ll Always Be Late (2019)']

In [228]:
len(search_results)

99

In [229]:
with open('../data/imdb_title_search_results.pickle', 'wb') as file:
    pickle.dump(search_results, file)

with open('../data/imdb_title_search_results_fails.pickle', 'wb') as file:
    pickle.dump(search_results_fails, file)